In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import torch as th
from syft.core.node.vm.plan_vm import PlanVirtualMachine
from syft import Plan

In [3]:
alice = PlanVirtualMachine(name="alice")
alice_client = alice.get_root_client()
remote_torch = alice_client.torch

# Option 1: make_plan

In [4]:
def make_plan(func):
    def func_wrapper(*args):
        vm = PlanVirtualMachine(name="alice")
        client = vm.get_root_client()
        inputs = [arg.send(client) for arg in args]    
        vm.record_actions()
        res = func(*inputs)
        vm.stop_recording()
        plan = Plan(actions=vm.recorded_actions, inputs=inputs, outputs=res)
        return plan.send(client)
    return func_wrapper

In [5]:
@make_plan
def test_plan_builder(inp):
    return inp + inp

In [6]:
input_tensor = th.tensor([1,2,3])

In [7]:
test_plan_builder

<function __main__.make_plan.<locals>.func_wrapper(*args)>

In [8]:
test_plan_pointer = test_plan_builder(input_tensor)

In [9]:
res = test_plan_pointer(input_tensor)

In [10]:
res.get()[0]

tensor([2, 4, 6])

# Option 2: @make_plan2 with PlanBuilder

In [11]:
class PlanBuilder():
    def __init__(self, vm):
        self.vm=vm
    
    def build(self, *args) -> 'PointerPlan':
        client = self.vm.get_root_client()
        inputs = [arg.send(client) for arg in args]    
        self.vm.record_actions()
        res = self.forward(*inputs)
        self.vm.stop_recording()
        plan = Plan(actions=self.vm.recorded_actions, inputs=inputs, outputs=res)
        return plan.send(client)

In [12]:
def make_plan2(func):
    res = PlanBuilder(vm=PlanVirtualMachine(name="alice"))
    res.forward=func
    return res

In [13]:
@make_plan2
def test_plan_builder2(inp):
    return inp + inp

In [14]:
test_plan_builder2

In [15]:
test_plan_pointer = test_plan_builder2.build(input_tensor)

In [16]:
res = test_plan_pointer(input_tensor)

[2021-03-01T10:47:11.050574+0100][CRITICAL][logger]][4819] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: cf0015db878c4ad3b2581b36a5ba10a7>.


In [17]:
res.get()[0]

tensor([2, 4, 6])

# Extra use case for PlanBuilder, wrapping state

In [18]:
model_pointer1 = th.tensor([1,2,3]).send(alice_client)
input_tensor = th.tensor([1,2,3])

In [19]:
class TestPlanBuilder(PlanBuilder):
    def __init__(self, vm, model_pointer):
        super().__init__(vm)
        self.model_pointer = model_pointer
    
    def forward(self, x):
        res = x * self.model_pointer
        return res

In [20]:
plan_pointer = TestPlanBuilder(alice, model_pointer1).build(input_tensor)

In [21]:
res = plan_pointer(input_tensor)

[2021-03-01T10:47:11.362435+0100][CRITICAL][logger]][4819] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 6baa1d04b677429c80b83d67f6b53817>.


In [22]:
res.get()[0]

tensor([1, 4, 9])